In [5]:
import json
import numpy as np
import pandas as pd

In [4]:
import sys
sys.path.append(r'C:/Users/vshnu/OneDrive/Desktop/Vishnuperson/RE/aspire_')
from connector import data_fetch_v2

product name

In [ ]:
prod = f"""
select user_id, prod_group, json_data, lender_id from sanction """
flag,prod_df, error_msg = data_fetch_v2(prod, host="13.126.97.63")

In [41]:
prod_df.head(4)

,user_id,lender_name,product
0,5828b717-bbd1-46fd-b579-4bc21cd2a0d8,liqui_loans,4M
1,41c99020-32a6-4911-afd3-313f08de9e03,liqui_loans,4M
2,a832e010-c86c-434c-8f3f-9c3d8c843617,liqui_loans,4M
3,1f2b7fe3-77e6-43e3-96aa-7cf85f7f838e,liqui_loans,4M


In [30]:
prod_df.dtypes

user_id        object
prod_group     object
b2b_partner    object
lender_name    object
product        object
dtype: object

In [10]:
prod_df["b2b_partner"] = prod_df['json_data'].apply(lambda x: json.loads(x).get("coupon_code_partner","") )

In [39]:
prod_df= prod_df.drop(columns=['b2b_partner','prod_group'])

In [17]:
prod_df["lender_name"] = prod_df["lender_id"].apply(lambda x: 'AVANTI' if x == "ed1f0ae3-95ce-4708-9dd9-f0dc98dba535" else
                                                    'Trillion loans' if x == "e4c85d41-2b12-4fd1-8230-b55df25f0925"  else
                                                    'aspire_finance' if x == "aab6357e-5f15-4182-b4f6-a9281a28e3ed" else
                                                     "liqui_loans" if x == "7feba239-fc0d-49a5-98e9-4fe10b40bc7a" else "other")

In [36]:
prod_df["product"] = prod_df['b2b_partner'].fillna(prod_df['prod_group'])

In [35]:
prod_df['b2b_partner']= prod_df['b2b_partner'].replace("", np.nan)

In [ ]:
prod_df.groupby('product')['user_id'].count()

Disbursal

In [ ]:
disb = f"""
select user_id, net_disbursed_amount, created_time, loan_start_date, utr, status from merchant_payments """
flag,disb_df, error_msg = data_fetch_v2(disb, host="13.126.97.63")

In [72]:
disb_df.head(3)

,user_id,net_disbursed_amount,utr,txn_date
0,41c99020-32a6-4911-afd3-313f08de9e03,310.0,None,2020-11
1,41c99020-32a6-4911-afd3-313f08de9e03,900.0,031820136692,2020-11
2,a832e010-c86c-434c-8f3f-9c3d8c843617,60.0,031921017922,2020-11


In [46]:
disb_df = disb_df[disb_df['status']=='success']

In [61]:
disb_df["txn_date"] = disb_df["loan_start_date"].fillna(disb_df["created_time"])

In [62]:
disb_df['txn_date'] = disb_df['txn_date'].astype(str).str[:7]


In [66]:
disb_df = disb_df.drop(columns=['loan_start_date','created_time'])

In [70]:
disb_df = disb_df.drop(columns= ['status'])

In [73]:
monthly_disb = disb_df.merge(prod_df, on= 'user_id', how='left')

In [77]:
monthly_disb = monthly_disb[monthly_disb["fl_disbursed"]==1]

In [75]:
import numpy as np

monthly_disb["fl_disbursed"] = np.where(
    ((monthly_disb["product"] == '4M') & (monthly_disb["utr"].notna())) | 
    (monthly_disb["product"] != '4M'),
    1, 
    0
)


In [83]:
monthly_disb["year"] =  disb_df['txn_date'].astype(str).str[:4]


C:\Users\vshnu\AppData\Local\Temp\ipykernel_22488\2496881396.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monthly_disb["year"] =  disb_df['txn_date'].astype(str).str[:4]


In [92]:
monthly_disb.dtypes

user_id                  object
net_disbursed_amount    float64
utr                      object
txn_date                 object
lender_name              object
product                  object
fl_disbursed              int64
year                     object
dtype: object

In [93]:
disbursal = round(monthly_disb.groupby(['product', 'year'])['net_disbursed_amount']
                  .sum()
                  .unstack(fill_value=0), 2)


In [98]:
import pandas as pd

pd.set_option('display.float_format', '{:}'.format)
print(disbursal)


year         2020        2021         2022         2023
product                                                
1M_OD         0.0         0.0          0.0   14085955.0
4M        34904.0 13018574.69 233500198.93 203940706.42
4M_0FEES      0.0         0.0    998748.68    572657.16
AGRI_1M       0.0         0.0          0.0      49580.0
ANX_1M        0.0         0.0         10.0          0.0
APNA          0.0         0.0     574750.0     519750.0
APNA_4M       0.0         0.0          0.0     107000.0
APNA_5M       0.0         0.0          0.0      32000.0
BIDSO_1M      0.0         0.0          0.0      13846.0
BZTM_1M       0.0         0.0         40.0     285105.0
FLPKRT_1M     0.0         0.0   9163749.81  27313456.13
NINJA_1M      0.0         0.0          0.0    4065978.2
PL            0.0         0.0          0.0  40645628.76
REDB_1M       0.0         0.0          0.0    181995.17
essmart       0.0         0.0          0.0       8400.0
yodacart      0.0         0.0          0.0     2